In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
a = torch.randn(2, 3)
print(a)
print(a.T)

tensor([[ 0.2359,  1.5187,  0.6454],
        [-0.9198,  2.0957, -0.2107]])
tensor([[ 0.2359, -0.9198],
        [ 1.5187,  2.0957],
        [ 0.6454, -0.2107]])


Generator

In [4]:
def yeild_smth(list):
    for i in list:
        yield i


list = [1, 2, 3]
gen = yeild_smth(list)

In [5]:
next(gen)

1

In [6]:
next(gen)

2

In [7]:
next(gen)

3

In [8]:
next(gen)

StopIteration: 

In [9]:
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

In [10]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        predictions = self.linear_relu_stack(x)
        return predictions


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)  # lr - learning rate
# SDG - stochastic gradient descent

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_nr, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_nr % 100 == 0:
            loss, current = loss.item(), batch_nr * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 1.661166  [    0/60000]
loss: 1.630442  [ 6400/60000]
loss: 1.483303  [12800/60000]
loss: 1.549213  [19200/60000]
loss: 1.442395  [25600/60000]
loss: 1.419811  [32000/60000]
loss: 1.430087  [38400/60000]
loss: 1.351956  [44800/60000]
loss: 1.392781  [51200/60000]
loss: 1.289518  [57600/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.314715 

Epoch 2
-------------------------------
loss: 1.388303  [    0/60000]
loss: 1.372744  [ 6400/60000]
loss: 1.208223  [12800/60000]
loss: 1.307764  [19200/60000]
loss: 1.193792  [25600/60000]
loss: 1.206669  [32000/60000]
loss: 1.216496  [38400/60000]
loss: 1.156069  [44800/60000]
loss: 1.201838  [51200/60000]
loss: 1.112193  [57600/60000]
Test Error: 
 Accuracy: 63.6%, Avg loss: 1.134316 

Epoch 3
-------------------------------
loss: 1.204225  [    0/60000]
loss: 1.207003  [ 6400/60000]
loss: 1.026124  [12800/60000]
loss: 1.157418  [19200/60000]
loss: 1.037960  [25600/60000]
loss: 1.064064  [32000/600

In [18]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

/tmp/ipykernel_53814/2085806346.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
